## goal of this notebook

train a collaborative filter on the Booklens dataset

In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
#path = "data/ml-20m/"
path = "data/book-lens/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

## set up data

In [4]:
ratings = pd.read_csv(path+'BX-Book-Ratings.csv', delimiter=';')
# I'm throwing away the 'implicit' ratings
ratings = ratings[ratings['Book-Rating'] != 0]
ratings.head(20)

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8


In [5]:
len(ratings)

433671

In [6]:
book_names = pd.read_csv(path+'BX-Books.csv', delimiter=';', error_bad_lines=False).set_index('ISBN')['Book-Title'].to_dict()

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: C

In [7]:
book_names.values()[:5]

['Dennis the Menace: Make Believe Angel (Fawcett Gold Medal Book)',
 'Losers, Inc.',
 'All That Counts',
 "Tess of the D'Urbervilles (Penguin Classics)",
 'How the Other Half Lives: Studies Among the Tenements of New York']

In [8]:
users = ratings['User-ID'].unique()
books = ratings['ISBN'].unique()
print(users[:5])
print(books[:5])

[276726 276729 276736 276737 276744]
['0155061224' '052165615X' '0521795028' '3257224281' '0600570967']


In [9]:
userid2idx = {o:i for i,o in enumerate(users)}
bookid2idx = {o:i for i,o in enumerate(books)}

In [10]:
ratings['ISBN'] = ratings['ISBN'].apply(lambda x: bookid2idx[x])
ratings['User-ID'] = ratings['User-ID'].apply(lambda x: userid2idx[x])

In [11]:
ratings.head(20)

,User-ID,ISBN,Book-Rating
1,0,0,5
3,1,1,3
4,1,2,6
6,2,3,8
7,3,4,6
8,4,5,7
9,5,6,10
16,6,7,9
19,6,8,9
20,6,9,8


In [113]:
from collections import Counter
from math import log

In [116]:
rating_counts = Counter(ratings['Book-Rating'])
print(rating_counts)
rating_weights = {score : 1.0 / counts * max(rating_counts.values()) for (score, counts) in rating_counts.iteritems()}
print(rating_weights)
log_rating_weights = {score : log(weight) for (score, weight) in rating_weights.iteritems()}
print(log_rating_weights)

Counter({8: 103736, 10: 78610, 7: 76457, 9: 67541, 5: 50974, 6: 36924, 4: 8904, 3: 5996, 2: 2759, 1: 1770})
{1: 58.607909604519776, 2: 37.59913011960855, 3: 17.300867244829885, 4: 11.650494159928122, 5: 2.0350767057715697, 6: 2.8094464305059037, 7: 1.3567887832376369, 8: 1.0, 9: 1.5358967145881761, 10: 1.3196285459865156}
{1: 4.0708696636608455, 2: 3.6269809150114343, 3: 2.8507566300062317, 4: 2.4553485962733395, 5: 0.7105335114380281, 6: 1.032987464118678, 7: 0.30512071895001197, 8: 0.0, 9: 0.4291143893579052, 10: 0.2773502924409179}


In [12]:
n_users = ratings['User-ID'].nunique()
n_books = ratings['ISBN'].nunique()
n_users, n_books

(77805, 185973)

In [13]:
# number of latent factors
n_fac = 50

In [14]:
#sample_size = 4000
sample_size = len(ratings)
msk = np.random.rand(sample_size) < 0.8
trn = ratings[:sample_size][msk]
val = ratings[:sample_size][~msk]

## define a linear model, with bias

This didn't work at all!

tried again with class_weights, still terrible. 20 epochs to hit ~12.5 val_loss

## define a neural network model

this is also not working well. keeps overfitting, and training accuracy is poor. benchmark is about 0.6 rmse, I'm at like 3.6! That's with a sample batch of 100_000, 20% validation, and this model:

input_users = Input(shape=(1,), dtype='int64', name='user_in')
user_embedding = Embedding(n_users, n_fac, input_length=1, W_regularizer=l2(1e-2))(input_users)
input_books = Input(shape=(1,), dtype='int64', name='book_in')
book_embedding = Embedding(n_books, n_fac, input_length=1, W_regularizer=l2(1e-2))(input_books)
x = merge([user_embedding, book_embedding], mode='concat')
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.8)(x)
x = Dense(1)(x)
nn = Model([input_users, input_books], x)
nn.compile(Adam(0.001), loss='mse')

Maybe try increasing model size and adding in more data to regularize?
Still pretty lousy. valid_loss is not beating ~12

Trying again with all the implicit (0) ratings removed. They are actually the majority!
This gives an enormous improvement. valid_loss ~ 2.7 after 3 epochs. But it never really improves. underfitting? yeah, looks like all predictions are quite similar.

So I try Jeremy Howard's trick of doing an initial slow epoch ( I have no idea why this would work ). Doesn't seem to help.

Someone on the forum recommends initializing by grossly overfitting a small sample. Worth a shot!
Nope

Ok, try increasing the model size more. 512 hidden units.

Nope, how about two layers of 2048? Nope.

Changing to RMSprop at least gives monotonically decreasing loss...

I should look at the unbalanced class problem, maybe set class weights.

In [117]:
input_users = Input(shape=(1,), dtype='int64', name='user_in')
user_embedding = Embedding(n_users, n_fac, input_length=1, W_regularizer=l2(1e-6))(input_users)
input_books = Input(shape=(1,), dtype='int64', name='book_in')
book_embedding = Embedding(n_books, n_fac, input_length=1, W_regularizer=l2(1e-6))(input_books)

In [118]:
x = merge([user_embedding, book_embedding], mode='concat')
x = Flatten()(x)
x = Dropout(0.0)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.0)(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.0)(x)
x = Dense(1)(x)
nn = Model([input_users, input_books], x)
nn.compile(RMSprop(0.0001), loss='mse')

In [126]:
nn.load_weights(model_path + 'pretrain.h5')

In [127]:
nn.optimizer.lr = 0.001

In [128]:
nn.fit([trn['User-ID'], trn['ISBN']], trn['Book-Rating'], batch_size=64, nb_epoch=3, 
          validation_data=([val['User-ID'], val['ISBN']], val['Book-Rating'])
      , class_weight=log_rating_weights)

Train on 346709 samples, validate on 86962 samples
Epoch 1/3
346709/346709 [==============================] - 249s - loss: 3.4494 - val_loss: 5.5893
Epoch 2/3
346709/346709 [==============================] - 249s - loss: 3.4545 - val_loss: 5.3517
Epoch 3/3
346709/346709 [==============================] - 248s - loss: 3.4457 - val_loss: 5.2692


In [129]:
# are all predictions the same?
print(nn.predict([np.array([3]), np.array([6])]))
print(nn.predict([np.array([3]), np.array([60])]))
print(nn.predict([np.array([3]), np.array([600])]))
print(nn.predict([np.array([30]), np.array([6])]))
print(nn.predict([np.array([300]), np.array([6])]))

[[ 5.6652]]
[[ 5.6652]]
[[ 5.6652]]
[[ 5.6652]]
[[ 5.6652]]
